In [1]:
# Import required libraries
import pandas as pd
import numpy as np 
import sklearn
import tensorflow as tf

# Import necessary modules
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#Keras specific
from tensorflow import keras
from tensorflow.keras import Input, Model, layers, Sequential
from tensorflow.keras.layers import Dense

from keras import models
from keras import layers

from keras.wrappers.scikit_learn import KerasClassifier

#Numpy specific
from numpy import mean
from numpy import absolute

tf.random.set_seed(3)

In [2]:
#Load in the data and seperate out columns
df = pd.read_csv('~/Downloads/DATA.csv')
df = df['STUDENT ID;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;COURSE ID;GRADE'].str.split(';', n=33, expand=True)

In [3]:
#Drop the first col as it is not useful for training
#Also convert all values within the dataframe to int since it recognized as string
df = df.drop(0, 1)
for column in df:
    df[column] = df[column].astype(int)

O:\Anaconda\envs\Tensorflow_Practice\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#Split original dataframe to parameter dataframe and target dataframe
target_column = [32] 
y = df[target_column].values

df = df.drop(32, 1)
predictors = list(df.columns)
X = df[predictors].values

O:\Anaconda\envs\Tensorflow_Practice\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [5]:
#Create the K fold model
cv = KFold(n_splits=10, shuffle=True, random_state = 15)

In [6]:
# Create function returning a compiled network 1
def create_network_1():
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=31, activation='relu', input_shape=(31,)))
    
    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=40, activation='relu'))
    
    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='tanh'))

    # Compile neural network
    network.compile(loss= 'MeanAbsoluteError', 
                    optimizer='adam')
    
    # Return compiled network
    return network


In [7]:
# Wrap Keras model so it can be used by scikit-learn
neural_network_1 = KerasClassifier(build_fn=create_network_1, epochs=100, verbose=0)

O:\Anaconda\envs\Tensorflow_Practice\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


In [8]:
# Evaluate neural network using ten-fold cross-validation
score_1 = cross_val_score(neural_network_1, X, y, scoring='neg_mean_absolute_error', cv=cv)
score_11 = cross_val_score(neural_network_1, X, y, scoring='accuracy', cv=cv)

In [9]:
#MAE score and accuracy for the first NN model
print("////////////////////////////MAE and Accuracy for the first NN///////////////////////////////////")
print(mean(absolute(score_1)))
print(mean(absolute(score_11)))

2.3290476190476186
0.24095238095238095


In [10]:
# Create function returning a compiled network
def create_network_2():
    # Start neural network
    network = models.Sequential()

    network.add(layers.Dense(units=31, activation='sigmoid', input_shape=(31,)))
    network.add(layers.Dense(units=100, activation='sigmoid'))
    network.add(layers.Dense(units=250, activation='sigmoid'))
    network.add(layers.Dense(units=1, activation='tanh'))

    # Compile neural network
    network.compile(optimizer = 'adam', loss= 'MeanAbsoluteError') 
    
    # Return compiled network
    return network


In [11]:
neural_network_2 = KerasClassifier(build_fn=create_network_2, 
                                 epochs=100, 
                                 verbose=0)

O:\Anaconda\envs\Tensorflow_Practice\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# Evaluate the second neural network using ten-fold cross-validation
score_2 = cross_val_score(neural_network_2, X, y, scoring='neg_mean_absolute_error', cv=cv)
score_22 = cross_val_score(neural_network_2, X, y, scoring='accuracy', cv=cv)

In [13]:
#MAE score and accuracy for the second NN model
print("////////////////////////////MAE and Accuracy for the second NN///////////////////////////////////")
print(mean(absolute(score_2)))
print(mean(absolute(score_22)))

2.3290476190476186
0.24095238095238095


In [14]:
"""
# Create function returning a compiled network
def create_network_opt(optimizer, activation_1, activation_2):
    # Start neural network
    network = models.Sequential()

    network.add(layers.Dense(units=31, activation=activation_1, input_shape=(31,)))
    network.add(layers.Dense(units=50, activation=activation_1))
    network.add(layers.Dense(units=8, activation=activation_2))

    # Compile neural network
    network.compile(loss='sparse_categorical_crossentropy', 
                    optimizer=optimizer, 
                    metrics=['accuracy']) 
    
    # Return compiled network
    return network
"""

"\n# Create function returning a compiled network\ndef create_network_opt(optimizer, activation_1, activation_2):\n    # Start neural network\n    network = models.Sequential()\n\n    network.add(layers.Dense(units=31, activation=activation_1, input_shape=(31,)))\n    network.add(layers.Dense(units=50, activation=activation_1))\n    network.add(layers.Dense(units=8, activation=activation_2))\n\n    # Compile neural network\n    network.compile(loss='sparse_categorical_crossentropy', \n                    optimizer=optimizer, \n                    metrics=['accuracy']) \n    \n    # Return compiled network\n    return network\n"

In [15]:
"""
neural_network_opt = KerasClassifier(build_fn=create_network_opt, verbose=0)
"""

'\nneural_network_opt = KerasClassifier(build_fn=create_network_opt, verbose=0)\n'

In [16]:
"""
#Using Grid search to obtain better result
#Partial code from https://chrisalbon.com/code/deep_learning/keras/tuning_neural_network_hyperparameters/
# Create hyperparameter space
epochs = [10, 100, 200]
batches = [100, 150, 200]
optimizers = ['rmsprop', 'adam', 'sgd', 'nadam']
activation_11 = ['tanh', 'sigmoid', 'relu']
activation_22 = ['tanh', 'sigmoid', 'relu']

# Create hyperparameter options
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, activation_1 = activation_11, activation_2 = activation_22)

# Create grid search
grid = GridSearchCV(estimator=neural_network_opt, cv=cv, param_grid=hyperparameters, scoring = 'neg_mean_absolute_error')

# Fit grid search
grid_result = grid.fit(X, y)
"""

"\n#Using Grid search to obtain better result\n#Partial code from https://chrisalbon.com/code/deep_learning/keras/tuning_neural_network_hyperparameters/\n# Create hyperparameter space\nepochs = [10, 100, 200]\nbatches = [100, 150, 200]\noptimizers = ['rmsprop', 'adam', 'sgd', 'nadam']\nactivation_11 = ['tanh', 'sigmoid', 'relu']\nactivation_22 = ['tanh', 'sigmoid', 'relu']\n\n# Create hyperparameter options\nhyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, activation_1 = activation_11, activation_2 = activation_22)\n\n# Create grid search\ngrid = GridSearchCV(estimator=neural_network_opt, cv=cv, param_grid=hyperparameters, scoring = 'neg_mean_absolute_error')\n\n# Fit grid search\ngrid_result = grid.fit(X, y)\n"

In [17]:
"""
print('Best Score: %s' % grid_result.best_score_)
print('Best Hyperparameters: %s' % grid_result.best_params_)
"""

"\nprint('Best Score: %s' % grid_result.best_score_)\nprint('Best Hyperparameters: %s' % grid_result.best_params_)\n"

In [18]:
#Optimized network after using grid search
def create_network_opt1():
    # Start neural network
    network = models.Sequential()

    network.add(layers.Dense(units=31, activation='relu', input_shape=(31,)))
    network.add(layers.Dense(units=50, activation='relu'))
    network.add(layers.Dense(units=8, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='sparse_categorical_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy']) 
    # Return compiled network
    return network

In [19]:
# Evaluate optimized neural network using ten-fold cross-validation
neural_network_opt1 = KerasClassifier(build_fn=create_network_opt1, batch_size = 100, epochs = 200, verbose=0)
score_opt = cross_val_score(neural_network_opt1, X, y, scoring='neg_mean_absolute_error', cv=cv)
score_opt2 = cross_val_score(neural_network_opt1, X, y, scoring='accuracy', cv=cv)

O:\Anaconda\envs\Tensorflow_Practice\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


In [20]:
#MAE score and accuracy for the optimized NN model
print("///////////////////////////////////////////////MAE and Accuracy for the optimized NN/////////////////////////////")
print(mean(absolute(score_opt)))
print(mean(absolute(score_opt2)))

1.6285714285714286
0.2138095238095238
